### Validating Areal Interpolation

The purpose of this notebook is to demonstrate that our areal interpolation functions produce extensive and intensive statistics similar to known values.

In [1]:
import sys

# others will need to change the below line to point at broadbandequity directory
# this is necessary so that the jupyternotebook can load our package
sys.path[0] = '/Users/drewkeller/Desktop/CS/broadbandequity'

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
from data_pipeline.fetch_census_data import acs5_aggregate 
from data_pipeline import spatial_operations as so
import numpy as np
import pandas as pd
from IPython.display import display

We will use ACS 5-year aggregate data from 2019 for our validation. We are comparing to known values from CMAP that also rely on aggregated ACS data:

"CCA values are estimated by aggregating ACS data for census tracts and block groups. Data from tracts and block groups located in multiple CCAs is allocated proportionally based on the block-level distribution of population, households or housing units (as appropriate) from the most recent Decennial Census."

Our approach:
1. Start with tract-level population data from ACS.
2. Calculate tract-level population density using tract shapefiles.
3. Aggregate tract-level population to community areas via areal-weighted sum.
4. Aggregate tract-level density to community areas via areal-weighted mean. Multiply by community-area area to get population.
5. Aggregate tract-level density to community areas via population-weighted mean. Multiply by community-area area to get population. _Note: Realized retroactively that this is invalid and results in upward bias - density inherently should be weighted by area, not population. To validate population-weighted mean we would need to use another known statistic like income._
6. Compare calculated populations to known values (Source: [CMAP: 2021 CDS based on 2019 population data](https://datahub.cmap.illinois.gov/dataset/community-data-snapshots-raw-data)).

In [3]:
# 1. Start with tract-level population data from ACS.
tract_data = acs5_aggregate()[["estimated total population","tract"]]
tract_data['population'] = tract_data['estimated total population']
tract_data = tract_data.drop(columns='estimated total population')
tract_data.head()

,tract,population
0,630200,1825
1,580700,5908
2,590600,3419
3,600700,2835
4,611900,1639


In [4]:
# 2. Calculate tract-level population density using tract shapefiles.
tract_data = so.geographize(tract_data,'tract')
tract_data["density"] = tract_data['population']/tract_data.area
tract_data.head()

,Shape_Leng,Shape_Area,SqMiles,GEOID2,tract,community_,geometry,population,area,density
0,37281.254752,6.177154e+07,0.000000,60840000,840000,60,"MULTIPOLYGON (((1096672.940 1927611.312, 10966...",2794.0,6.177154e+07,0.000045
1,6102.900390,1.777035e+06,0.000000,76840801,840801,76,"MULTIPOLYGON (((1096672.956 1927611.220, 10966...",NaN,1.777035e+06,NaN
2,16035.054991,8.947394e+06,0.320945,59840300,840300,59,"POLYGON ((1163591.927 1881471.238, 1163525.437...",3511.0,8.947394e+06,0.000392
3,14719.012184,8.946045e+06,0.320896,60840200,840200,60,"POLYGON ((1172724.229 1887341.263, 1172726.397...",2419.0,8.946045e+06,0.000270
4,15186.400644,1.230614e+07,0.441424,34841100,841100,34,"POLYGON ((1176041.550 1889791.988, 1176043.375...",7142.0,1.230614e+07,0.000580


In [5]:
# 3. Aggregate tract-level population to community areas via areal-weighted sum.
community_pop = so.aggregate(tract_data,{'population' : 'areal sum'},'community_area','tract')
community_pop.head()

ValueError: You have multiple data points for the same geographical unit. Combine these data points and try again.

In [ ]:
# 4. Aggregate tract-level density to community areas via areal-weighted mean. Multiply by community-area area to get population.
community_density_areal = so.geographize(so.aggregate(tract_data,{'density' : 'areal mean'},'community_area','tract'),'community_area')
community_density_areal['population'] = community_density_areal['density']*community_density_areal['area']
community_density_areal = community_density_areal[['community_area','population']]
community_density_areal.head()

In [ ]:
# 5. Aggregate tract-level density to community areas via population-weighted mean. Multiply by community-area area to get population.
community_density_pop = so.geographize(so.aggregate(tract_data,{'density' : 'pop mean'},'community_area','tract'),'community_area')
community_density_pop['population'] = community_density_pop['density']*community_density_pop['area']
community_density_pop = community_density_pop[['community_area','population']]
community_density_pop.head()

In [ ]:
# 6. Compare calculated populations to known values 

# first step: load validation data
validation_data = pd.read_csv('../data/CMAP_2019_comm_data.csv')[['GEOG','TOT_POP']]
validation_data['GEOG'] = [str(i).upper() for i in validation_data['GEOG']]
validation_data = validation_data.rename(columns={'GEOG':'community_area','TOT_POP':'known population'})
validation_data['community_area'] = validation_data['community_area'].replace({"O'HARE":"OHARE","THE LOOP": "LOOP"})
validation_data = validation_data.dropna()
validation_data.head()

In [ ]:
# second step: place calculated and known values side-by-side with errors
community_pop = community_pop.rename(columns={'population':'areal-weighted sum'})
community_density_areal = community_density_areal.rename(columns={'population':'areal-weighted mean'})
community_density_pop = community_density_pop.rename(columns={'population':'pop-weighted mean'})
validation_data = validation_data.join(community_pop.set_index('community_area'),on='community_area')
validation_data['areal-weighted sum error'] = validation_data['areal-weighted sum']-validation_data['known population']
validation_data = validation_data.join(community_density_areal.set_index('community_area'),on='community_area')
validation_data['areal-weighted mean error'] = validation_data['areal-weighted mean']-validation_data['known population']
validation_data = validation_data.join(community_density_pop.set_index('community_area'),on='community_area')
validation_data['pop-weighted mean error'] = validation_data['pop-weighted mean']-validation_data['known population']
validation_data.head()

In [ ]:
# fourth step: add simple crosswalk

crosswalk = pd.read_csv("../data/chicago_internet.csv")[['name','total_pop']]
crosswalk['name'] = [str(i).upper() for i in crosswalk['name']]
crosswalk = crosswalk.rename(columns={'name':'community_area','total_pop':'crosswalk'})
crosswalk = crosswalk.replace({"O'HARE":"OHARE"})
validation_data = validation_data.join(crosswalk.set_index('community_area'),on='community_area')
validation_data['crosswalk error'] = validation_data['crosswalk']-validation_data['known population']

In [ ]:
# third step: stats

print('Areal-weighted sum:')
print(f'Maximum error: {max(abs(validation_data["areal-weighted sum error"]))}')
print(f'Median error: {np.median(validation_data["areal-weighted sum error"])}')
print(f'Mean error: {np.mean(validation_data["areal-weighted sum error"])}')
print(f'RMS error: {np.sqrt(np.average(validation_data["areal-weighted sum error"]**2))}')
outliers5 = sum([1 if abs(i)>0.05 else 0 for i in validation_data['areal-weighted sum error']/validation_data['known population']])
outliers20 = sum([1 if abs(i)>0.2 else 0 for i in validation_data['areal-weighted sum error']/validation_data['known population']])
print(f'Community areas off by more than 5%, 20%: {outliers5},{outliers20}')
print('')

print('Areal-weighted mean:')
print(f'Maximum error: {max(abs(validation_data["areal-weighted mean error"]))}')
print(f'Median error: {np.median(validation_data["areal-weighted mean error"])}')
print(f'Mean error: {np.mean(validation_data["areal-weighted mean error"])}')
print(f'RMS error: {np.sqrt(np.average(validation_data["areal-weighted mean error"]**2))}')
outliers5 = sum([1 if abs(i)>0.05 else 0 for i in validation_data['areal-weighted mean error']/validation_data['known population']])
outliers20 = sum([1 if abs(i)>0.2 else 0 for i in validation_data['areal-weighted mean error']/validation_data['known population']])
print(f'Community areas off by more than 5%, 20%: {outliers5},{outliers20}')
print('')

print('Pop-weighted mean:')
print(f'Maximum error: {max(abs(validation_data["pop-weighted mean error"]))}')
print(f'Median error: {np.median(validation_data["pop-weighted mean error"])}')
print(f'Mean error: {np.mean(validation_data["pop-weighted mean error"])}')
print(f'RMS error: {np.sqrt(np.average(validation_data["pop-weighted mean error"]**2))}')
outliers5 = sum([1 if abs(i)>0.05 else 0 for i in validation_data['pop-weighted mean error']/validation_data['known population']])
outliers20 = sum([1 if abs(i)>0.2 else 0 for i in validation_data['pop-weighted mean error']/validation_data['known population']])
print(f'Community areas off by more than 5%, 20%: {outliers5},{outliers20}')
print('')

print('Simple crosswalk:')
print(f'Maximum error: {max(abs(validation_data["crosswalk error"]))}')
print(f'Median error: {np.median(validation_data["crosswalk error"])}')
print(f'Mean error: {np.mean(validation_data["crosswalk error"])}')
print(f'RMS error: {np.sqrt(np.average(validation_data["crosswalk error"]**2))}')
outliers5 = sum([1 if abs(i)>0.05 else 0 for i in validation_data['crosswalk error']/validation_data['known population']])
outliers20 = sum([1 if abs(i)>0.2 else 0 for i in validation_data['crosswalk error']/validation_data['known population']])
print(f'Community areas off by more than 5%, 20%: {outliers5},{outliers20}')

Discussion:

Areal-weighted sum and mean produce very similar results, as expected. (Actually, in many cases, precisely the same results.) Both are within a few hundred residents of our validation data for most neighborhoods, but both have a handful of neighborhoods that they underestimate by up to 10,000 residents.

This is definitely cause for concern, but perhaps for a more complicated reason than the aggregation function not working, as the low median error and clustering of most results near the known value seem to suggest that the aggregation function likely works (unless the overlapping-measurement part is broken and the outliers have more overlapping tracts?).

Population-weighted mean overestimates _most_ neighborhoods by several thousand residents. On reflection, this is actually expected behavior because it's not really valid to take a population-weighted mean of population density; it makes sense that this would bias population results upwards. In other words, these results suggest the population-weighted mean function is working, but to confirm we may need to use a different validation variable.

The simple crosswalk method - one-to-one from tracts to community areas - performs best by a signficant amount. In fact, a majority (48/77) community areas obtain the exact value from this method! 

This does raise some questions. Are those 48/77 community areas ones in which tract boundaries align exactly? If so this result could make sense, although it would then be concerning why the aggregation function also didn't give exact results. The next step may be to inspect one of these community areas in depth to see if we can identify what is going on.

In [ ]:
# save to CSV
validation_data.to_csv('validation_data')

In [ ]:
# display in full
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(validation_data.round().convert_dtypes(convert_integer=True))